<a href="https://colab.research.google.com/github/solomontessema/building-agentic-ai/blob/main/notebooks/2.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Prompt Flow Refactor</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

intent_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    Classify the intent of this query: {query}
    Categories: weather, news, search, database, unknown.
    Just return the category.
    """
)
intent_chain = intent_prompt | llm | StrOutputParser()


followup_prompt = ChatPromptTemplate.from_template(
    template="""
    Category: {{category}}
    {%if constraint %}
    Please ensure to follow this rule: {{constraint}}
    {% endif %}
    Write the category followed by a follow-up question for clarification.
    """,
    template_format="jinja2"
)

followup_chain = followup_prompt | llm | StrOutputParser()

pipeline = (
    RunnableParallel(
        category=intent_chain,
        constraint=RunnablePassthrough(),
    )
    | followup_chain
)

output = pipeline.invoke({"query": "What is happening in China right now?", "constraint": "Limit response strictly to 5 words."})
print(output)

### Assginment

Try queries such as:

"What’s the weather in Nairobi?"

"Summarize the latest news on GPT-4"

You can also test with constraints like:

"Use simple language"

"Limit response to 2 sentences"

"Focus on business context"

Observe how the pipeline adapts.
